In [4]:
import pandas as pd

agency = pd.read_csv('../datasets/asana/gtfs_data/agency.txt')
routes = pd.read_csv('../datasets/asana/gtfs_data/routes_utf8.txt')
stop_times = pd.read_csv('../datasets/asana/gtfs_data/stop_times.txt')
stops = pd.read_csv('../datasets/asana/gtfs_data/stops.txt')
trips = pd.read_csv('../datasets/asana/gtfs_data/trips.txt')

In [6]:
modeling_df = pd.DataFrame()
# By the nature of the GTFS static data structure, this is only a "Scheduled-based arrival time estimation”

route_travel_time = pd.DataFrame()
arrival_time = pd.DataFrame()
delay_prediction = pd.DataFrame()
# most descriptive columns from the gtfs format for modeling that i currently identified are:
#  from stop_times : stop_id, arrival_time, departure_time
#  from stops : stop_id, stop_lat, stop_lon
#  from trips : shape_id, route_id (To relate the info, basically a join table)

# The features that i imagine i could extract to get more info are:

# average_stop_times (Would describe an average of time from stop to stop per route)
# a more defined and specific attributes below:

#avg_seconds_between_stops
#std_seconds_between_stops

# hour_of_day (Pretty descriptive name)
# stop_quantity (A sum of the bus stops)
# stop_density_500m (This measures how conglomerated a zone is in the 500m radious)


gps_data = pd.read_csv('../datasets/asana/gps_data.csv') # thing is so big i need to put attributes here because checking takes long - id,deviceid,devicetime,latitude,longitude,speed,route_direction,route_id,route_guid,device_guid

# Fordward here im going to explain the structure of how this dataset is going to be analized

# I'm going to add steps to it: 
# 1st Step is going to ananlyze just structural, coordinates, stop density, etc. So i can see for example if using a delay in time target (Which will probably be the most used target in this notebook), i can see how much the data is giving in comparison 
# ------- (Here on step 1, will be using normal train_test_split and will be trained with diferent models and compared with CV)
# to 2nd Step which will only use context data such as speeds, times, etc. || i will search for most significant/descriptive attributes for each, and look into compare important stadistic values such as R^2 and others
# ------- (Here on step 2, will be using time_series_split with lags, so the model also finds relationship in between the difference in time)
# Finally on step 3, depending on which attributes tend to be the most meaningful and tune the best models to find a in between in overfitting and underfitting 


# Now onto other other targets its going to be the effective_speeds
# Using the GTFS-RT, finding the actual time taken to complete a route, we can find the delays in the actual schedulet time, which will be useful to create a "isDelayed" target using a specific treshold
# Clustering could be applied using the times, the route and locations and stop densities to find congestion zones

